In [1]:
import pandas as pd
import numpy as np
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel
import gensim
from gensim.utils import simple_preprocess

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
reviews_df = pd.read_csv('/Users/sudeshgurung/Msc-Data_Analytics/ADTA_with_python/coursework_part_2/Part_2/dataset/amazon_cells_labelled.txt', sep="\t" , names = ['Review', 'Sentiment'])
reviews_df.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


## Pre-processing

In [3]:
reviews_df = reviews_df.astype({'Review': 'string'})
reviews_df.dtypes

Review       string
Sentiment     int64
dtype: object

In [4]:
# only the sentence for text pre-processing 
reviews = reviews_df['Review']
reviews[:5]

0    So there is no way for me to plug it in here i...
1                          Good case, Excellent value.
2                               Great for the jawbone.
3    Tied to charger for conversations lasting more...
4                                    The mic is great.
Name: Review, dtype: string

In [5]:
reviews1 = []

for row in reviews: 
    #remove punctuation 
    row = row.translate(str.maketrans('', '', string.punctuation))
    
    #remove digits/numbers 
    row = row.translate(str.maketrans('', '', string.digits))
    
    #change to lowercase 
    row = row.lower()
    
    reviews1.append(row)
reviews1

['so there is no way for me to plug it in here in the us unless i go by a converter',
 'good case excellent value',
 'great for the jawbone',
 'tied to charger for conversations lasting more than  minutesmajor problems',
 'the mic is great',
 'i have to jiggle the plug to get it to line up right to get decent volume',
 'if you have several dozen or several hundred contacts then imagine the fun of sending each of them one by one',
 'if you are razr owneryou must have this',
 'needless to say i wasted my money',
 'what a waste of money and time',
 'and the sound quality is great',
 'he was very impressed when going from the original battery to the extended battery',
 'if the two were seperated by a mere  ft i started to notice excessive static and garbled sound from the headset',
 'very good quality though',
 'the design is very odd as the ear clip is not very comfortable at all',
 'highly recommend for any one who has a blue tooth phone',
 'i advise everyone do not be fooled',
 'so far 

In [6]:
reviews1 = pd.Series(reviews1)
reviews1[:5]

0    so there is no way for me to plug it in here i...
1                            good case excellent value
2                                great for the jawbone
3    tied to charger for conversations lasting more...
4                                     the mic is great
dtype: object

In [7]:
# setting english stopwords
stop_words = set(stopwords.words('English'))

reviews_temp = reviews1.apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))
reviews_temp[:10]

0                      way plug us unless go converter
1                            good case excellent value
2                                        great jawbone
3    tied charger conversations lasting minutesmajo...
4                                            mic great
5         jiggle plug get line right get decent volume
6    several dozen several hundred contacts imagine...
7                                   razr owneryou must
8                            needless say wasted money
9                                     waste money time
dtype: object

In [8]:
tokens =[]
for doc in reviews1: 
    temp = word_tokenize(doc)
    tokens.append(temp)
# tokens



In [9]:
result = []
for doc in tokens: 
    x = [i for i in doc if not i in stop_words]
    result.append(" ".join(x))
result = pd.Series(result)
result[:10]

0                      way plug us unless go converter
1                            good case excellent value
2                                        great jawbone
3    tied charger conversations lasting minutesmajo...
4                                            mic great
5         jiggle plug get line right get decent volume
6    several dozen several hundred contacts imagine...
7                                   razr owneryou must
8                            needless say wasted money
9                                     waste money time
dtype: object

In [10]:
# removing character less than 3
removing_lt_3 = result.apply(lambda x: ' '.join(term for term in x.split() if len(term) > 2))
removing_lt_3

0                              way plug unless converter
1                              good case excellent value
2                                          great jawbone
3      tied charger conversations lasting minutesmajo...
4                                              mic great
                             ...                        
995           screen get smudged easily touches ear face
996                          piece junk lose calls phone
997                                   item match picture
998                 thing disappoint infra red port irda
999                       answer calls unit never worked
Length: 1000, dtype: object

In [11]:
# apply lemmatizing with POS tags 
from nltk.corpus import wordnet 
from nltk.stem import WordNetLemmatizer

def get_wordnet_pos(word):
    # Map POS tag to first character Lemmatize() accepts 
    tag = nltk.pos_tag([word])[0][1][0].upper()
#     display(tag)
    tag_dict = {
        "J": wordnet.ADJ,
        "N": wordnet.NOUN, 
        "V": wordnet.VERB,
        "R": wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

# init WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
reviews3 = removing_lt_3.apply(lambda x: ' '.join(lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in x.split()))
reviews3[:10] 

0                            way plug unless converter
1                            good case excellent value
2                                        great jawbone
3    tie charger conversation last minutesmajor pro...
4                                            mic great
5         jiggle plug get line right get decent volume
6    several dozen several hundred contact imagine ...
7                                   razr owneryou must
8                               needle say waste money
9                                     waste money time
dtype: object

In [12]:
#putting back preprocessed text and labels 

temp = list(zip(reviews3, reviews_df['Sentiment']))
reviews2 = pd.DataFrame(temp, columns = ['Review', 'Sentiment'])
reviews2

,Review,Sentiment
0,way plug unless converter,0
1,good case excellent value,1
2,great jawbone,1
3,tie charger conversation last minutesmajor pro...,0
4,mic great,1
...,...,...
995,screen get smudge easily touch ear face,0
996,piece junk lose call phone,0
997,item match picture,0
998,thing disappoint infra red port irda,0


In [13]:
pip install bertopic

  Using cached PyYAML-5.4.1-cp39-cp39-macosx_10_9_x86_64.whl (259 kB)
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install bert-tensorflow

Note: you may need to restart the kernel to use updated packages.


In [15]:
from bertopic import BERTopic 

In [16]:
# from sentence_transformers import SentenceTransformer
# from umap import UMAP
# from hdbscan import HDBSCAN

# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
# umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
# hdbscan_model = HDBSCAN(min_cluster_size=80, min_samples=40,
#                         gen_min_span_tree=True,
#                         prediction_data=True)

In [17]:
# from sklearn.feature_extraction.text import CountVectorizer

from nltk.corpus import stopwords

stopwords = list(stopwords.words('english'))

# # we add this to remove stopwords that can pollute topcs
# # vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)
vectorizer_model = TfidfVectorizer(ngram_range=(1, 1), stop_words=stopwords)

In [18]:
data = reviews2.Review.values.tolist()
data

['way plug unless converter',
 'good case excellent value',
 'great jawbone',
 'tie charger conversation last minutesmajor problem',
 'mic great',
 'jiggle plug get line right get decent volume',
 'several dozen several hundred contact imagine fun send one one',
 'razr owneryou must',
 'needle say waste money',
 'waste money time',
 'sound quality great',
 'impressed go original battery extend battery',
 'two seperated mere start notice excessive static garble sound headset',
 'good quality though',
 'design odd ear clip comfortable',
 'highly recommend one blue tooth phone',
 'advise everyone fool',
 'far good',
 'work great',
 'click place way make wonder long mechanism would last',
 'go motorolas website follow direction could get pair',
 'bought use kindle fire absolutely love',
 'commercial mislead',
 'yet run new battery two bar thats three day without charge',
 'bought mother problem battery',
 'great pocket phone combination',
 'ive own phone month say best mobile phone ive',
 

In [19]:
# from bertopic import BERTopic

# model = BERTopic(
#     umap_model=umap_model,
#     hdbscan_model=hdbscan_model,
#     embedding_model=embedding_model,
#     vectorizer_model=vectorizer_model,
#     language='english',
#     calculate_probabilities=True,
#     verbose=True
# )
# topics, probs = model.fit_transform(data)

In [20]:
topic_model = BERTopic(
    vectorizer_model=vectorizer_model,
    language='english', 
    calculate_probabilities=True,
    verbose=True
)

In [ ]:
topics, probabilities = topic_model.fit_transform(data)

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.get_topic(0)

In [ ]:
topic_model.visualize_barchart(top_n_topics=10)

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
topic_model.visualize_distribution(probabilities[0])

In [ ]:
topic_model.visualize_distribution(probabilities[1])